In [62]:
import re
from pprint import pprint

with open("/workspace/fit-perf_logs.txt", "r") as f:
    log_text = "".join(f.readlines())
# Sample log text
# Function to parse a single block
def parse_block(block_text):
    profile_info = {}
    
    # First line: Profile stats for: [LightningModule]OrientedRCNN.configure_callbacks
    match = re.search(r"Profile stats for: (.+)", block_text)
    if match:
        profile_info['profile_for'] = match.group(1)
        
    # Second line: 7 function calls in 0.000 seconds
    match = re.search(r"(\d+) function calls in ([\d.]+) seconds", block_text)
    if match:
        profile_info['function_calls'] = int(match.group(1))
        profile_info['total_time'] = float(match.group(2))
        
    # Table parsing
    table_lines = re.findall(r"\d+\s+[\d.]+\s+[\d.]+\s+[\d.]+\s+[\d.]+\s+.+", block_text)
    function_list = []
    for line in table_lines:
        elements = re.split(r"\s+", line.strip())
        function_info = {
            'ncalls': int(elements[0]),
            'tottime': float(elements[1]),
            'percall_1': float(elements[2]),
            'cumtime': float(elements[3]),
            'percall_2': float(elements[4]),
            'filename_function': ' '.join(elements[5:])
        }
        function_list.append(function_info)
    profile_info['functions'] = function_list
    
    return profile_info

# Function to parse the whole log file
def parse_log(log_text):
    # Split the log text into blocks based on the "Profile stats for:" line
    blocks = re.split(r"(Profile stats for: .+)", log_text)[1:]
    parsed_blocks = []
    for i in range(0, len(blocks), 2):
        block_text = blocks[i] + blocks[i + 1]
        parsed_block = parse_block(block_text)
        parsed_blocks.append(parsed_block)
    return parsed_blocks

# Test the function
parsed_log = parse_log(log_text)


In [63]:
sorted_parsed_log = sorted(parsed_log, key=lambda x: sum(y['cumtime'] for y in x['functions']), reverse=True)

In [64]:
temp = {}
for row in sorted_parsed_log:
    if len(temp) < 20:
        temp[row['profile_for']] = temp.get(row['profile_for'], 0) + sum(y['cumtime'] for y in row['functions'])
    else:
        break

In [65]:
temp = sorted(temp.items(), key=lambda x: x[1], reverse=True)

In [66]:
temp

[('[LightningModule]OrientedRCNN.optimizer_step', 1983282.7829999987),
 ('run_training_epoch', 1719111.251000001),
 ('[Strategy]SingleDeviceStrategy.training_step', 1003381.9020000006),
 ('run_training_batch', 980564.062),
 ('[Strategy]SingleDeviceStrategy.validation_step', 493627.24700000207),
 ('[LightningModule]RotatedFasterRCNN.optimizer_step', 344409.42800000013),
 ('[_TrainingEpochLoop].train_dataloader_next', 274895.0989999998),
 ('[Strategy]SingleDeviceStrategy.backward', 141576.745),
 ('[_EvaluationLoop].val_next', 133223.91799999995),
 ('[Strategy]SingleDeviceStrategy.batch_to_device', 69462.73999999995),
 ('[LightningModule]OrientedRCNN.configure_gradient_clipping',
  43989.782000000014),
 ('[LightningModule]OrientedRCNN.on_validation_epoch_end', 26057.001000000026),
 ('[LightningModule]OrientedRCNN.transfer_batch_to_device',
  15016.253000000002),
 ("[Callback]ModelCheckpoint{'monitor': None, 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval